## **Sun Zi's Perfect Math Class**

### 🔎 Phân tích đề bài

- Cho `e = 3, c1, c2, c3, n1, n2, n3` là các số lớn, tìm m thỏa mãn `c = m^e (mod n)`
### 🎯 Định hướng giải bài
- Áp dụng CRT để giải ta có:
- `m^e = c1 (mod n1)`, `m^e = c2 (mod n2)`, `m^e = c2 (mod n2)`
- từ định lý thặng dư trung hoa ta tìm được `m^e`, do e nhỏ nên có thể tính căn để lấy giá trị `m`
- ta nên để ý thực hiện phép nhân thay vì phép chia vì số lớn khó thực hiện phép chia
- muốn khai căn số lớn chính xác ta sử dụng `gmpy2.iroot()`

In [1]:
import functools
import operator
from Crypto.Util.number import long_to_bytes, inverse
import gmpy2
e = 3
c1 = 105001824161664003599422656864176455171381720653815905925856548632486703162518989165039084097502312226864233302621924809266126953771761669365659646250634187967109683742983039295269237675751525196938138071285014551966913785883051544245059293702943821571213612968127810604163575545004589035344590577094378024637
c2 = 31631442837619174301627703920800905351561747632091670091370206898569727230073839052473051336225502632628636256671728802750596833679629890303700500900722642779064628589492559614751281751964622696427520120657753178654351971238020964729065716984136077048928869596095134253387969208375978930557763221971977878737
c3 = 64864977037231624991423831965394304787965838591735479931470076118956460041888044329021534008265748308238833071879576193558419510910272917201870797698253331425756509041685848066195410586013190421426307862029999566951239891512032198024716311786896333047799598891440799810584167402219122283692655717691362258659
n1 = 147896270072551360195753454363282299426062485174745759351211846489928910241753224819735285744845837638083944350358908785909584262132415921461693027899236186075383010852224067091477810924118719861660629389172820727449033189259975221664580227157731435894163917841980802021068840549853299166437257181072372761693
n2 = 95979365485314068430194308015982074476106529222534317931594712046922760584774363858267995698339417335986543347292707495833182921439398983540425004105990583813113065124836795470760324876649225576921655233346630422669551713602423987793822459296761403456611062240111812805323779302474406733327110287422659815403
n3 = 95649308318281674792416471616635514342255502211688462925255401503618542159533496090638947784818456347896833168508179425853277740290242297445486511810651365722908240687732315319340403048931123530435501371881740859335793804194315675972192649001074378934213623075830325229416830786633930007188095897620439987817

# Bước 1: tìm N = n1 * n2 * n3
N = n1 * n2 * n3
# Cách khác là dùng vòng for hoặc reduce(operator, list) của thư viện functools cùng operator
N_reduce = functools.reduce(operator.mul, [n1, n2, n3])

cs = [c1, c2, c3]
ns = [n1, n2, n3]

# Bước 2: ta tìm x thỏa mãn: x bằng sum(ci * N / (ni) * inverse(N, ni)) % N với inverse là nghịch đảo module
# tuy nhiên do là số lớn nên phép chia không hoạt động, ta đổi N/(ni) = tích các nj với j khác i
crt = c1 * n2 * n3 * inverse(n2 * n3, n1)
crt += c2 * n1 * n3 * inverse(n1 * n3, n2)
crt += c3 * n1 * n2 * inverse(n1 * n2, n3)
x = crt%N

# kiểm tra điều kiện
assert x % n1 == c1
assert x % n2 == c2
assert x % n3 == c3
# Bước 3: x = m^e % N, do đó m^e = x + k*N ta cần tìm m^e thỏa mãn khai căn được và có flag đúng, ở đây ta sử dụng gmpy2 với iroot để khai căn số lớn
for k in range(0, 5):
    # hàm trả về kết quả khai căn và biến đánh giá khai căn có đúng hay không
    m,check = gmpy2.iroot(x + k*N,3)
    # assert check
    if (check == True):
        print(long_to_bytes(m))


b'DUCTF{btw_y0u_c4n_als0_us3_CRT_f0r_p4rt14l_fr4ct10ns}'


# **decrypt then eval**

### 🔎 Phân tích đề bài

- Bài sử dụng mã hóa AES với KEY và IV được sinh ngẫu nhiên, FLAG mặc định.
- Input đầu vào là giá trị hex, sau đó chương trình chuyển giá trị hex này thành byte.
- Đối tượng `aes` được tạo để thực hiện mã hóa **hoặc** giải mã (không thể dùng chung một đối tượng `aes` cho cả hai).
- Lỗi xuất hiện ở câu lệnh `eval()`, khi câu lệnh này thực thi lệnh được truyền vào ở dạng chuỗi.

### 🎯 Định hướng giải bài

- **Khai thác lỗi `eval()`**: Mục tiêu là truyền vào `eval()` lệnh `"print(FLAG)"`. Để đạt được điều này, ta cần:
    - `aes.decrypt(ct) = b"print(FLAG)"`
    - `ct = aes.encrypt(b"print(FLAG)")`
    - Từ đó suy ra `input = aes.encrypt(b"print(FLAG)").hex()` (lưu ý `aes` phải được tạo riêng là `AES.new(KEY, AES.MODE_CFB, IV, segment_size=128)`)
- **Vấn đề gặp phải**: Chưa tìm được cách thực hiện lệnh `AES.new(KEY, AES.MODE_CFB, IV, segment_size=128).encrypt(b"print(FLAG"))` khi nhập input.
- **Bế tắc**:
    - Không thể chuyển cả đoạn code trên thành hex vì ý nghĩa khác nhau. Ta muốn chuyển đổi output của đoạn code đó sang hex, chứ không phải các ký tự trong code.
    - Một số ý tưởng như in ra KEY và IV không khả thi vì KEY và IV không thay đổi trong một phiên làm việc, nhưng mọi lệnh đều phải được mã hóa bằng KEY và IV.


In [17]:
'''code test of decrypt then eval'''
from Crypto.Cipher import AES
import os

# chọn giữ nguyên KEY và IV để test
KEY = b'\xe8\x7f\xaf\x96\x7f\x14V\x8d)\xb6Y7\xcd\xec\xf1\xb9'
IV = b'\x8a(v\xc2\xb1h\x90\x82\xcb\xa71\x8c\x9a\xbe\xe7\xf5'
# chọn flag là hello để test
FLAG = 'hello'

def main():
    # thử ngiệm input là aes.encrypt(b'print(FLAG)').hex(), với biểu diễn dữ liệu là hex, thông tin là kết quả của hàm aes.encrypt(b'print(FLAG)')
    inputs = AES.new(KEY, AES.MODE_CFB, IV, segment_size=128).encrypt(b'print(FLAG)').hex()
    # cho ct chuyển hex thành bytes, lúc này ct = aes.encrypt(b'print(FLAG)')
    ct = bytes.fromhex(inputs)
    # thực hiện giải mã ct, sau đó thực hiện eval
    aes = AES.new(KEY, AES.MODE_CFB, IV, segment_size=128)
    print(eval(aes.decrypt(ct)))
if __name__ == '__main__':
    main()

hello
None


In [21]:
def oracle(ct):
    # returns True if eval succeeded
    conn.sendlineafter(b'ct: ', ct.hex().encode())
    o = conn.recvline(timeout=1)
    return o is None or ('invalid ct' not in o.decode())

conn = remote('2024.ductf.dev', 30020)

# đoạn này nhiệm vụ là tìm kí tự đầu tiên của flag, với cách là thử từng kí tự từ 0 đến 255, nếu 7 kết quả cuối cùng đều là True thì dừng
# lí do 7 true thì thỏa mãn là bởi vì ta dùng aes để mã hóa với 128 bit, nên mỗi kí tự sẽ được mã hóa bằng 7 bit đó
results = []
for b in range(256):
    cand = bytes([b])
    results.append(oracle(cand))
    if results[-1] == True:
        print(cand)
    if set(results[-8:]) == {True}:
        break
z = len(results) - 1

[x] Opening connection to 2024.ductf.dev on port 30020
[x] Opening connection to 2024.ductf.dev on port 30020: Trying 34.87.243.24
[+] Opening connection to 2024.ductf.dev on port 30020: Done
b'F'
b'G'
b'H'
b'I'
b'J'
b'K'
b'L'
b'M'


In [23]:
# test sự hợp lệ của eval()
for i in range(256):
    try:
        print(eval(bytes([i])))
        print(bytes([i]))
    except:
        continue

0
b'0'
1
b'1'
2
b'2'
3
b'3'
4
b'4'
5
b'5'
6
b'6'
7
b'7'
8
b'8'
9
b'9'
1
b'_'
77
b'b'
105
b'i'
77
b'z'
